# Conversão de Metadados de Produtos de Ecommerce Para Representação Vetorial com MetaProd2Vec

O objetivo deste notebook é, passo a passo, implementar a conversão dos dados em uma representação vetorial (Embeddings)
<br/>
<br/>
<br/>

##### Instalando dependências

Se já executou o comando <code>pip install -r requirements.txt</code>, sugerido no arquivo README.md,
não a necessidade de instalar novamente

In [5]:
# ## Remova o comentário para instalar as dependências #### #

# %pip install tensorflow tensorboard pandas plotly pandas-profiling

##### Instalando o dataset

In [6]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Software.json.gz

'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [1]:
import tensorflow as tf
import tensorboard as tb
import plotly
import pandas as pd
import json
import gzip
from pandas_profiling import ProfileReport
# from vespa.application import Vespa

##### Conectando ao Vespa

Referências em <https://blog.vespa.ai/run-search-engine-experiments-in-Vespa-from-python/>

In [7]:
# app = Vespa(url = "https://testap-container.testaap.teste.aws-us-east-1c.dev.z.vespa-app.cloud/")

In [20]:
data = []
with gzip.open('./datasets/meta_Software.json.gz') as file:
    for linha in file:
        data.append(json.loads(linha.strip()))
    
print(len(data))
print("\n\n\n")
print(data[0])

26790




{'category': [], 'tech1': '', 'description': [], 'fit': '', 'title': 'HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK CONCEPTS', 'also_buy': [], 'tech2': '', 'brand': 'HOLT. RINEHART AND WINSTON', 'feature': [], 'rank': '25,550 in Software (', 'also_view': [], 'main_cat': 'Software', 'similar_item': '', 'date': '</div>', 'price': '.a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right;margin-top:1em}#aloha-cart-popover .deviceContainer{width:160px;float:left;padding-right:10px;border-right:1px solid #ddd}#aloha-cart-popover li:last-child{border-right:0}#aloha-cart-popover .aloha-device-title{height:3em;overflow:hidden}#aloha-cart-popover .aloha-thumbnail-container{height:100px

In [3]:
df = pd.DataFrame.from_dict(data)
print(len(df))

26790


#### Tutorial de projeção na documentação

<https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin?hl=pt-br>

In [10]:
# embeddings = MetaProd2Vec(dados)
# td.projection(embeddings)

In [11]:
# Criando o prifile
profile = ProfileReport(df, title='Relatório - Pandas Profiling', html={'style': {'full_width': True}})

In [23]:
# Salvando o profile
caminho_arquivo = "./relatorios/Relatorio01.html"
profile.to_file(output_file=caminho_arquivo)

NameError: name 'profile' is not defined

In [24]:
import webbrowser
import os

webbrowser.open(f"file://{os.path.realpath(caminho_arquivo)}")

True

## Processamento com MetaProd2Vec

Pegando informações que importam

In [5]:
sessions = []
for title, df in df.groupby('title', sort=False):
  session = []
  for index, row in df.iterrows():
    session.append('price: {}'.format(row['price']))
    session.append('title: {}'.format(row['title']))
    session.append('category: {}'.format(row['category']))
    
  sessions.append(session)

In [6]:
' '.join(sessions[0])

'price: .a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right;margin-top:1em}#aloha-cart-popover .deviceContainer{width:160px;float:left;padding-right:10px;border-right:1px solid #ddd}#aloha-cart-popover li:last-child{border-right:0}#aloha-cart-popover .aloha-device-title{height:3em;overflow:hidden}#aloha-cart-popover .aloha-thumbnail-container{height:100px;margin-bottom:1em;text-align:center}#aloha-cart-popover .aloha-price-container{text-align:center}#aloha-cart-popover .aloha-thumbnail-container img{height:inherit}#aloha-cart-popover .aloha-cart{border-top:1px solid #ddd;border-bottom:1px solid #ddd}#aloha-cart-popover #aloha-cart-info{margin-right:0}#alohaBuyBoxWidget .without-co

## Treinando

In [103]:
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):
  '''Callback to save model after each epoch.'''

  def __init__(self, path_prefix):
   self.path_prefix = path_prefix
   self.epoch = 0

  def on_epoch_end(self, model):
   print(f"Epoch #{self.epoch}")
   self.epoch += 1

  def on_train_end(self, model):
   output_path = f'./models/{self.path_prefix}.model'
   model.save(output_path)

In [104]:
import multiprocessing
from gensim.models import Word2Vec

saver = EpochSaver('MP2V')

model = Word2Vec(sessions,
    # size=128,           # Vector dimensions
    window=10,          # Sliding window size
    sg=1,               # Use the skip gram model
    hs=0,               # Use negative sampling
    negative=20,        # Number of negative samples
    ns_exponent=-0.5,   # Unigram distribution's exponent
    sample=1e-4,        # Subsampling rate
    workers=(2 * multiprocessing.cpu_count() + 1),
    callbacks=[saver],
    # iter=10)
)

Epoch #0
Epoch #1
Epoch #2
Epoch #3
Epoch #4


In [105]:
from collections import defaultdict

def most_similar(price, category, title, model, *args, **kwargs):
  result = model.wv.most_similar(*args, **kwargs)
  return result

In [106]:
print(df.head()['title'])

26789    CNC Designs - Vector Graphics for Computer Con...
Name: title, dtype: object


In [107]:
title = df['title']
category = df['category']
price = df['price']

In [108]:
termo_de_pesquisa = "Cyberlink PowerDirector 13 Ultimate Suite"
categoria_pesquisa = "title"

In [109]:
coisas_relacionadas = most_similar(title, category, price, model, f'{categoria_pesquisa}: {termo_de_pesquisa}', topn=20)

In [110]:
for coisa in coisas_relacionadas:
    print(coisa)

("category: ['Software', 'Video', 'Video Editing', '</span></span></span>']", 0.9997012615203857)
('price: \n\t\t\t\t\t\t\t\t\t\t\t\t<span class="verticalAlign a-size-large"', 0.9991224408149719)
('category: [\'Software\', "Children\'s", \'Early Learning\']', 0.9990817308425903)
('category: [\'Software\', "Children\'s", \'Games\', \'</span></span></span>\']', 0.9990806579589844)
('price: ', 0.9990490674972534)
("category: ['Software', 'Lifestyle &amp; Hobbies', 'Cooking &amp; Health']", 0.9990406036376953)
('category: [\'Software\', "Children\'s", \'Art & Creativity\']', 0.9990343451499939)
('category: [\'Software\', "Children\'s", \'Thinking & Problem Solving\']', 0.9989576935768127)
('price: \n\t\t                                            \n\r\n\r\n\r\n\r\n\r\n    \r\n    \r\n        \r\n        \n\n\n\n\n\n\n\n\n\n\n\n    \n    \n    \n       \n           <div class="a-section a-spacing-none"', 0.9989050626754761)
("category: ['Software', 'Lifestyle & Hobbies', 'Home Publishing', 